<a href="https://colab.research.google.com/github/chizuchizu/IOAI/blob/main/Task2/chizu_100_task2_vis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
# ====================================================
# CFG
# ====================================================

class CFG:
    num_workers=4
    project = "IOAI_Task2_classification"
    name = "chizu_023_task2_dropout"

    # model
    base_model_name = "ioai2024japan/chizu_015_task2_redrock005_base"
    tokenizer_name = "ioai2024japan/redrock_005_task2_tokenizer"
    num_classes = 5
    dropout = 0.3

    # training
    epochs = 20

    scheduler='linear' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']

    lr = 1e-05

    # dataset
    max_length = 256

    # T4: 32
    # L4: 64
    train_batch_size = 64
    eval_batch_size = 64

    seed=42
    train=True

# for wandb
cfg = dict(vars(CFG))
cfg = {k: v for k, v in cfg.items() if "__" not in k}

In [3]:
from google.colab import userdata

read_access_token = userdata.get('hf_read')
write_access_token = userdata.get('hf_write')

### Dependencies

In [3]:
import importlib
import torch, transformers

if '2.3.0' not in torch.__version__:
  !pip install torch==2.3.0
if transformers.__version__!='4.41.2':
  !pip install transformers==4.41.2

if importlib.util.find_spec('datasets') is None:
  !pip install datasets==2.18.0s
  !pip install evaluate==0.4.2
  !pip install accelerate -U


ERROR: Could not find a version that satisfies the requirement datasets==2.18.0s (from versions: 0.0.9, 1.0.0, 1.0.1, 1.0.2, 1.1.0, 1.1.1, 1.1.2, 1.1.3, 1.2.0, 1.2.1, 1.3.0, 1.4.0, 1.4.1, 1.5.0, 1.6.0, 1.6.1, 1.6.2, 1.7.0, 1.8.0, 1.9.0, 1.10.0, 1.10.1, 1.10.2, 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 1.13.2, 1.13.3, 1.14.0, 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.17.0, 1.18.0, 1.18.1, 1.18.2, 1.18.3, 1.18.4, 2.0.0, 2.1.0, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.3.2, 2.4.0, 2.5.0, 2.5.1, 2.5.2, 2.6.0, 2.6.1, 2.6.2, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.10.0, 2.10.1, 2.11.0, 2.12.0, 2.13.0, 2.13.1, 2.13.2, 2.14.0, 2.14.1, 2.14.2, 2.14.3, 2.14.4, 2.14.5, 2.14.6, 2.14.7, 2.15.0, 2.16.0, 2.16.1, 2.17.0, 2.17.1, 2.18.0, 2.19.0, 2.19.1, 2.19.2, 2.20.0)
ERROR: No matching distribution found for datasets==2.18.0s
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 5.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

If you've just installed `accelerate`, execute `Runtime > Restart session and run all` in the Colab UI menu above.

In [4]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.functional import F

import torchvision
from torchvision import datasets, transforms, models

from tqdm.auto import tqdm

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, get_scheduler
from datasets import load_dataset, Dataset, DatasetDict

import evaluate

from huggingface_hub import login

login(token=read_access_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [50]:
classification_dataset = load_dataset(
    'InternationalOlympiadAI/NLP_problem',
    token=read_access_token,
    )
tokenizer = AutoTokenizer.from_pretrained(CFG.tokenizer_name)

def transform(example_batch):
    example_batch["text"] = [transliterate_brahmi_to_latin(x) for x in example_batch["text"]]
    inputs = tokenizer([x for x in example_batch["text"]],  truncation=True, max_length=CFG.max_length, padding="max_length", return_tensors="pt")
    inputs["labels"] = example_batch["label"]
    inputs["raw"] = example_batch["text"]
    return inputs


tokenized_data = classification_dataset.with_transform(transform)

model = AutoModelForSequenceClassification.from_pretrained(
    CFG.base_model_name, num_labels=CFG.num_classes, token=read_access_token,
    output_hidden_states=True,
)

config.json:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [6]:
# tokenized_data["train"][0]

In [35]:
train_loader = DataLoader(
    tokenized_data["train"],
    batch_size=1,
    num_workers=0,
    pin_memory=True,
    shuffle=False,
    drop_last=True,
)

In [8]:
token_embeddings = model.get_input_embeddings().weight.clone()

In [9]:
vectors = {}
vocab = tokenizer.get_vocab()
for idx in vocab.values():
    vectors[idx] = token_embeddings[idx].detach().numpy().copy()

In [10]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)

In [11]:
np.array(list(vectors.values())).shape

(52000, 768)

In [20]:
reduced_vectors = tsne.fit_transform(np.array(list(vectors.values()))[:500, :])

In [21]:
import pandas as pd
df = pd.DataFrame(reduced_vectors, columns=['x', 'y'])
df['token'] = list(vocab.keys())[:500]
df.head()

,x,y,token
0,-36.110752,-15.281016,खतथचल
1,-25.679972,-28.954433,मचभचढच
2,-16.978527,11.147544,डबष००
3,-3.180740,-27.463448,०बढनह
4,1.573361,2.832579,झचणणचथचथथनड


In [22]:
import plotly.express as px

# Plotlyで可視化
fig = px.scatter(df, x='x', y='y',  text='token',
                 title='BERT Token Embeddings Visualization with t-SNE and Clustering')
fig.update_traces(textposition='top center')
fig.show()

In [52]:
# トークン埋め込みを取得する関数
def get_token_embeddings(inputs):
    del inputs["raw"]
    outputs = model(**inputs)

    last_hidden_states = outputs.hidden_states[-1]
    return last_hidden_states[0].detach().numpy()


# 全てのテキストの埋め込みを取得
all_embeddings = []
labels = []
texts = []
i = 0
for inputs in tqdm(train_loader):
    labels.append(inputs["labels"][0])
    texts.append(inputs["raw"][0])
    embeddings = get_token_embeddings(inputs)
    # print(embeddings.shape)
    all_embeddings.append(embeddings)


    i += 1
    if i == 500:
        break


  0%|          | 0/1524 [00:00<?, ?it/s]

In [66]:
# トークン埋め込みを取得する関数
def get_token_embeddings(inputs):
    del inputs["raw"]
    outputs = model(**inputs)

    last_hidden_states = outputs.hidden_states[-1]
    return last_hidden_states[0].detach().numpy(), outputs.logits[0].argmax(-1).detach().numpy()


# 全てのテキストの埋め込みを取得
all_embeddings = []
preds = []
labels = []
texts = []
i = 0
for inputs in tqdm(train_loader):
    labels.append(inputs["labels"][0])
    texts.append(inputs["raw"][0])
    embeddings, pred = get_token_embeddings(inputs)
    # print(embeddings.shape)
    all_embeddings.append(embeddings)
    preds.append(int(pred))


    i += 1
    if i == 500:
        break


  0%|          | 0/1524 [00:00<?, ?it/s]

In [41]:
np.array(all_embeddings).shape

(100, 256, 768)

In [67]:
tsne = TSNE(n_components=2, random_state=42)
embeddings_2d = tsne.fit_transform(np.array(all_embeddings).reshape(500, -1))

In [70]:
df = pd.DataFrame(embeddings_2d, columns=['x', 'y'])
df['text'] = [i for i in range(len(texts))]
# df['text'] = texts#[text[:4] for text in texts]

df['cluster'] = [int(x) for x in labels]
df["pred"] = preds
df['correct'] = [int(x) == y for x, y in zip(labels, preds)]

df.head()

,x,y,text,cluster,pred,correct
0,10.204849,9.442760,0,0,0,True
1,15.432344,-15.046974,1,4,4,True
2,-6.648195,18.043077,2,3,3,True
3,9.063275,-14.187074,3,4,4,True
4,14.592372,-16.788172,4,4,4,True


In [63]:
fig = px.scatter(df, x='x', y='y', color='cluster', text='text',
                 title='BERT Text Embeddings Visualization with t-SNE')
fig.update_traces(textposition='top center')
fig.show()

In [69]:
fig = px.scatter(df, x='x', y='y', color='pred', text='text',
                 title='BERT Text Embeddings Visualization with t-SNE')
fig.update_traces(textposition='top center')
fig.show()

In [71]:
fig = px.scatter(df, x='x', y='y', color='correct', text='text',
                 title='BERT Text Embeddings Visualization with t-SNE')
fig.update_traces(textposition='top center')
fig.show()

In [ ]:
!pip install bertopic -q
!pip install umap-learn -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 92.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 10.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.


In [ ]:
import pandas as pd
from umap import UMAP
from bertopic import BERTopic
from IPython.display import display
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:

# %%
token_embeddings = model.get_input_embeddings().weight.clone()
vocab = tokenizer.get_vocab()
vectors = {}


# %%
for idx in vocab.values():
    vectors[idx] = token_embeddings[idx].detach().numpy().copy()


# %%
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)


# %%
reduced_vectors = tsne.fit_transform(list(vectors.values())[:3000])


# %%
import holoviews as hv
from holoviews import opts

hv.extension('plotly')


# %%
points = hv.Points(reduced_vectors)
labels = hv.Labels({('x', 'y'): reduced_vectors, 'text': [token for token, _ in zip(vocab, reduced_vectors)]}, ['x', 'y'], 'text')

(points * labels).opts(
    opts.Labels(xoffset=0.05, yoffset=0.05, size=14, padding=0.2, width=1500, height=1000),
    opts.Points(color='black', marker='x', size=3),
)

In [ ]:
tokenized_dataset = classification_dataset

In [ ]:
# topicやprobabilityの計算
topics, probs = topic_model.fit_transform(df_docs["data"])

df_docs["topic_predicted"] = topics
display(df_docs)

# 各Topicの概要
display(topic_model.get_topic_info())

# 各dataの可視化
topic_model.visualize_documents(df_docs["data"].values)

In [24]:


brahmi_to_devanagari = {
    '𑀓': 'क', '𑀔': 'ख', '𑀕': 'ग', '𑀖': 'घ', '𑀗': 'ङ', '𑀘': 'च', '𑀙': 'छ',
    '𑀚': 'ज', '𑀛': 'झ', '𑀜': 'ञ', '𑀝': 'ट', '𑀞': 'ठ', '𑀟': 'ड', '𑀠': 'ढ',
    '𑀡': 'ण', '𑀢': 'त', '𑀣': 'थ', '𑀤': 'द', '𑀥': 'ध', '𑀦': 'न', '𑀧': 'प',
    '𑀨': 'फ', '𑀩': 'ब', '𑀪': 'भ', '𑀫': 'म', '𑀬': 'य', '𑀭': 'र', '𑀮': 'ल',
    '𑀯': 'व', '𑀰': 'श', '𑀱': 'ष', '𑀲': 'स', '𑀳': 'ह', '𑁦':'०', '𑁣': 'नब्बे'
}

def transliterate_brahmi_to_latin(text):
    transliterated_text = ''
    for char in text:
        if char in brahmi_to_devanagari:
            transliterated_text += brahmi_to_devanagari[char]
        else:
            transliterated_text += char
    return transliterated_text


f1 = evaluate.load("f1")


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return f1.compute(predictions=predictions, references=labels, average='macro')

def to_device(batch, device):
    output = {}
    for k, v in batch.items():
        try:
            output[k] = v.to(device)
        except:
            output[k] = v
    return output


class Focal_MultiLabel_Loss(nn.Module):
    def __init__(self, gamma):
      super(Focal_MultiLabel_Loss, self).__init__()
      self.gamma = gamma
      # self.bceloss = nn.BCELoss(reduction='none')

    def forward(self, outputs, targets):
      # bce = self.bceloss(outputs, targets)
      ce = F.cross_entropy(outputs, targets)
      bce_exp = torch.exp(-ce)
      focal_loss = (1-bce_exp)**self.gamma * ce
      return focal_loss.mean()

In [ ]:
def main():

    wandb.init(
        name=CFG.name,
        project=CFG.project,
        config=cfg
    )
    classification_dataset = load_dataset(
        'InternationalOlympiadAI/NLP_problem',
        token=read_access_token,
        )
    tokenizer = AutoTokenizer.from_pretrained(CFG.tokenizer_name)

    def transform(example_batch):
        example_batch["text"] = [transliterate_brahmi_to_latin(x) for x in example_batch["text"]]
        inputs = tokenizer([x for x in example_batch["text"]],  truncation=True, max_length=CFG.max_length, padding="max_length", return_tensors="pt")
        inputs["labels"] = example_batch["label"]
        return inputs


    tokenized_data = classification_dataset.with_transform(transform)

    model = AutoModelForSequenceClassification.from_pretrained(
        CFG.base_model_name, num_labels=CFG.num_classes, token=read_access_token,
        hidden_dropout_prob=CFG.dropout,
    ).cuda()



    # dataset
    train_dataset = tokenized_data["train"]
    eval_dataset = tokenized_data["dev"]
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.train_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=True,
        drop_last=True,
    )
    eval_loader = DataLoader(
        eval_dataset,
        batch_size=CFG.eval_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=False,
    )

    epochs = CFG.epochs
    num_training_steps = epochs * len(train_loader)
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr)
    scheduler = get_scheduler(name=CFG.scheduler, optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)
    focal = Focal_MultiLabel_Loss(gamma=2)


    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    def train_one_epoch(model, scheduler, train_loader, optimizer):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, dynamic_ncols=True)

        for step, batch in enumerate(progress_bar):
            batch = to_device(batch, device)
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                token_type_ids=batch["token_type_ids"],
                labels=batch["labels"],
            )
            loss = outputs.loss
            # loss = focal(outputs.logits, batch["labels"])
            loss.backward()

            text = f"step {step}, loss: {loss:.5f}"
            progress_bar.set_description(text)

            wandb.log(
                {
                    "train_loss": loss,
                    "lr": optimizer.param_groups[0]["lr"],
                    "step": step,
                }
            )
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

    def evaluate_model(model, test_loader):
        model.eval()
        for batch in eval_loader:
            batch = to_device(batch, device)
            with torch.no_grad():
                outputs = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    token_type_ids=batch["token_type_ids"],
                    labels=batch["labels"],
                )

            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
        return f1.compute(predictions=predictions, references=batch["labels"], average='macro')

    # Train and evaluate the model
    model.to(device)
    for i in range(CFG.epochs):
        train_one_epoch(model, scheduler, train_loader, optimizer)
        accuracy = evaluate_model(model, eval_loader)
        wandb.log(
            {
                "epoch": i+1,
                "accuracy": accuracy
            }
        )
        print(f'Epoch {i+1} {accuracy}')
    return model

In [ ]:
model = main()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ioai2024japan/Task2_chizu_007_pretrain and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/23 [00:00<?, ?it/s]

Epoch 1 {'f1': 0.08}


  0%|          | 0/23 [00:00<?, ?it/s]

In [ ]:
model.push_to_hub(
    f"ioai2024japan/{CFG.name}",
    token=userdata.get('hf_write'), private=True
)

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ioai2024japan/chizu_022_task2_linear/commit/0b1ed2402f520290908e454abf57d825ac91ec54', commit_message='Upload BertForSequenceClassification', commit_description='', oid='0b1ed2402f520290908e454abf57d825ac91ec54', pr_url=None, pr_revision=None, pr_num=None)